# Selecting copper providers

Kernel: `bw25`

In [ ]:
import bw2data as bd
import bw2calc as bc
import bw_processing as bwp
import numpy as np

Ecoinvent and base data already imported

In [ ]:
if 'ei38-teaching-25' not in bd.projects:
    bi.restore_project_directory("/srv/data/projects/ecoinvent38-25.tar.gz")

In [ ]:
bd.projects.set_current('ei38-teaching-25')

In [ ]:
tox = ('ReCiPe Midpoint (E) V1.13', 'human toxicity', 'HTPinf')

2.5 gets rid of the `mapping` dictionary, using instead the SQLite row ids. It exposes this value with the `.id` property.

In [ ]:
bd.databases

In [ ]:
car = bd.get_activity(database="ei 3.8 cutoff", name='transport, passenger car, electric')
car

In [ ]:
fu, data_objs, _ = bd.prepare_lca_inputs({car: 1}, method=tox)

See [bw_processing](https://github.com/brightway-lca/bw_processing) for a description of these data packages.

In [ ]:
lca = bc.LCA(fu, data_objs=data_objs)

In [ ]:
lca.lci()
lca.lcia()

In [ ]:
lca.score

# Modifying the supply chain

Let's test the impact of knowing specifically where our copper comes from

In [ ]:
[(o.id, o.key) for o in bd.Database("ei 3.8 cutoff") if o['name'] == 'market for copper concentrate, sulfide ore']

In [ ]:
copper = bd.get_activity(database="ei 3.8 cutoff", name='market for copper concentrate, sulfide ore')
copper

This is a market, with different mines (and their varying technologies, concentrations, and energy sources) over the world contributing.

In [ ]:
sum(exc['amount'] for exc in copper.technosphere())

The tox impact per kilogram of copper varies a lot, though some of this is due to allocation across multiple metals.

In [ ]:
for exc in copper.technosphere():
    lca.redo_lcia({exc.input.id: 1})
    print(lca.score, exc.input)

# Modification approach 1: Create new LCA for each possibility

In [ ]:
possibles = [exc.input for exc in copper.technosphere()]

In [ ]:
possibles

Our approach here is simple: We want to set the inputs to the market to zero to everything except the one input we are considering:

In [ ]:
def create_replacement_vector_dp(parent, possibles, selected):
    modified = bwp.create_datapackage()
    # Everything is zero...
    data = np.zeros(len(possibles))
    indices = np.zeros(len(possibles), dtype=bwp.INDICES_DTYPE)
    
    for index, obj in enumerate(possibles):
        if obj.id == selected:
            # ... except the one input we selected. Minus sign because it is consumed.
            # Could also be positive, and then use a `flip` vector. See 
            # bw_processing and matrix_utils for more details.
            data[index] = -1
        indices[index] = (obj.id, parent)
        
    modified.add_persistent_vector(
        matrix="technosphere_matrix",
        indices_array=indices,
        name="Substitute global copper mix",
        data_array=data,
    )
    return modified

To use this new substituting data package, we just add it to the list of other data packages (but at the end, so it has the final word).

In [ ]:
for supplier in possibles:
    dp = create_replacement_vector_dp(copper.id, possibles, supplier.id)
    
    lca = bc.LCA(fu, data_objs=data_objs + [dp])
    lca.lci()
    lca.lcia()
    
    print(lca.score, supplier)

# Modification approach 2: Array of possible suppliers

We can reuse the LCA object (though the technosphere matrix will be automatically regenerated) we use arrays instead of vectors.

This is an evolution of the [presamples](https://github.com/PascalLesage/presamples/) approach. One big change versus `presamples` is that we no can add whatever rows, columns, or values we want to the matrix, without needing to create dummy values to be modified later.

In [ ]:
def create_replacement_array_dp(parent, possibles):
    modified = bwp.create_datapackage(sum_intra_duplicates=True, sum_inter_duplicates=False)
    data = np.diag(np.ones(len(possibles))) * -1
    indices = np.zeros(len(possibles), dtype=bwp.INDICES_DTYPE)
    
    for index, obj in enumerate(possibles):
        indices[index] = (obj.id, parent)
                
    modified.add_persistent_array(
        matrix="technosphere_matrix",
        indices_array=indices,
        name="Substitute global copper mix",
        data_array=data,
    )
    return modified

Arrays lets us express multiple scenarios or system configurations in one data format. Each column is a different configuration.

In [ ]:
np.diag(np.ones(10)) * -1

When using these configurations, we can just iterate through the LCA object with `next`. The iteration order for each data package is controlled by its [policies](https://github.com/brightway-lca/bw_processing#policies).

In [ ]:
lca = bc.LCA(fu, data_objs=data_objs + [create_replacement_array_dp(copper.id, possibles)], use_arrays=True)
lca.lci()
lca.lcia()

for supplier in possibles:
    print(lca.score, supplier)
    next(lca)

The correct approach for you will depend on your problem, how imperatirve or functional you are feeling that day, your mood, etc ;)